In [1]:
import numpy as np
import os
from sklearn.metrics import mean_squared_error as mse_f
from scipy import sparse
from scipy.stats import gamma
from scipy.stats import ttest_ind
import warnings
import pandas as pd

warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


In [14]:
paper_model_names = models = {
    "dsbmm_dpf.z-only": "DPIF-DD-Topic+Net",
    "dsbmm_dpf.z-theta-joint": "DPIF-DD-Topic+Net+Pubs",
    # 'spf.main':'MSPF',
    "unadjusted.main": "Unadjusted",
    "network_pref_only.main": "Network Only",
    "topic_only.main": "DPIF-DD-Topic",
    "no_unobs.main": "Oracle",
    "topic_only_oracle.main": "DPIF-Topic (No region)",
}


In [6]:
tmp_kv = tuple(paper_model_names.items())
for k, v in tmp_kv:
    if "dsbmm_dpf" in k:
        paper_model_names[k + "-ndc"] = v + " (No DC)"

tmp_dict = {
    k + "old_subs" + "_ewcnone" + "_rcolmain_adm1_1hot": v
    for k, v in paper_model_names.items()
    if "dsbmm_dpf" not in k
}
sub_choice_pretty = {"old_subs": " (prev. subs.) ", "upd_subs": " (subs. w. trans.) "}
reg_choice_pretty = {"adm1": "(ADM1)", "ctry": "(Ctry.)"}
for k, v in paper_model_names.items():
    if "dsbmm_dpf" in k:
        for sub_choice in ["old_subs", "upd_subs"]:
            for region in ["adm1", "ctry"]:
                tmp_dict[k + f"{sub_choice}_ewcnone_rcolmain_{region}_1hot"] = (
                    v + sub_choice_pretty[sub_choice] + reg_choice_pretty[region]
                )


In [8]:
full_paper_model_names = tmp_dict

In [9]:
def print_table(exp_results, regimes, models, exps=10):
    ncols = len(regimes.keys())
    nrows = len(models)
    results = np.zeros((nrows, ncols))
    std = np.zeros((nrows, ncols))

    col_idx = 0
    for regime, c in regimes.items():
        row_idx = 0
        for model in models:
            mse = np.zeros(exps)
            for i in range(exps):
                try:
                    beta_predicted = exp_results[c][model][i][0]
                    truth = exp_results[c][model][i][1]
                    sq_err = (beta_predicted - truth) ** 2
                    mse[i] = sq_err.mean()
                except:
                    print(model, "exp", i, "not found")
            results[row_idx][col_idx] = round(mse.mean() * 1000, 2)
            std[row_idx][col_idx] = round(mse.std() * 1000, 2)
            row_idx += 1
        col_idx += 1

    proper_names = [full_paper_model_names[m] for m in models]
    col_names = list(regimes.keys())
    df = pd.DataFrame(results, index=proper_names, columns=col_names, dtype=str)
    std_df = pd.DataFrame(std, index=proper_names, columns=col_names, dtype=str)
    df = df + "$\pm$" + std_df
    return df


### Load results

In [13]:
list(full_paper_model_names.keys())[:5]

['unadjusted.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'network_pref_only.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'topic_only.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'no_unobs.mainold_subs_ewcnone_rcolmain_adm1_1hot',
 'topic_only_oracle.mainold_subs_ewcnone_rcolmain_adm1_1hot']

In [4]:
from pathlib import Path

res_dir = Path("/scratch/fitzgeraldj/data/caus_inf_data/results")
exps = 5
embed = "user"
sub_choice = "old_subs"  # or "upd_subs" for dsbmm_dpf models
region = "adm1"  # or "ctry" for dsbmm_dpf models
base_models = [
    "unadjusted.main",
    "network_pref_only.main",
    # "topic_only.main",
    # "no_unobs.main",
    # "topic_only_oracle.main",
    "dsbmm_dpf.z-only",
    "dsbmm_dpf.z-theta-joint",
    "dsbmm_dpf.z-theta-joint-ndc",
]
models = [m + f"{sub_choice}_ewcnone_rcolmain_{region}_1hot" for m in base_models]

conf_types = ["homophily", "exog", "both"]
confounding_strengths = [(50, 10), (50, 50), (50, 100)]
exp_results = {}

for i in range(1, exps + 1):
    for model in models:
        for (cov1conf, cov2conf) in confounding_strengths:
            for ct in conf_types:
                try:
                    base_file_name = (
                        "conf="
                        + str((cov1conf, cov2conf))
                        + ";conf_type="
                        + ct
                        + ".npz"
                    )
                    result_file = (
                        (res_dir / str(i)) / model + "_model_fitted_params"
                    ) / base_file_name
                    res = np.load(result_file)
                    params = res["fitted"]
                    truth = res["true"]

                    if (ct, (cov1conf, cov2conf)) in exp_results:
                        if model in exp_results[(ct, (cov1conf, cov2conf))]:
                            exp_results[(ct, (cov1conf, cov2conf))][model].append(
                                (params, truth)
                            )
                        else:
                            exp_results[(ct, (cov1conf, cov2conf))][model] = [
                                (params, truth)
                            ]
                    else:
                        exp_results[(ct, (cov1conf, cov2conf))] = {
                            model: [(params, truth)]
                        }
                except:
                    print(result_file, " not found")


### Visualize results

In [ ]:
confounding_type = "exog"
models = list(exp_results[(confounding_type, confounding_strengths[1])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df1 = print_table(exp_results, regime1, models)


In [ ]:
confounding_type = "homophily"
models = list(exp_results[(confounding_type, confounding_strengths[0])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df2 = print_table(exp_results, regime1, models)


In [ ]:
confounding_type = "both"
models = list(exp_results[(confounding_type, confounding_strengths[0])].keys())
regime1 = {
    "Low": (confounding_type, confounding_strengths[0]),
    "Med.": (confounding_type, confounding_strengths[1]),
    "High": (confounding_type, confounding_strengths[2]),
}

df3 = print_table(exp_results, regime1, models)


In [9]:
all_results = pd.concat([df1, df2, df3], axis=1, keys=["Exog.", "Homophily", "Both"])
all_results


,Low,Med.,High,Low,Med.,High,Low,Med.,High
BD Adj.,0.17$\pm$0.07,0.17$\pm$0.16,0.2$\pm$0.17,0.29$\pm$0.06,0.27$\pm$0.08,0.25$\pm$0.06,0.13$\pm$0.06,0.13$\pm$0.09,0.12$\pm$0.06
Unadjusted,1.56$\pm$0.21,2.0$\pm$0.35,2.16$\pm$0.31,1.91$\pm$0.25,2.4$\pm$0.4,2.44$\pm$0.33,2.21$\pm$0.25,2.49$\pm$0.35,2.56$\pm$0.35
MSPF,0.53$\pm$0.42,0.56$\pm$0.35,1.1$\pm$1.55,0.66$\pm$0.38,0.67$\pm$0.37,0.76$\pm$0.48,0.55$\pm$0.34,0.62$\pm$0.43,1.5$\pm$2.67
Network Only,0.48$\pm$0.14,0.84$\pm$0.22,0.84$\pm$0.25,0.76$\pm$0.2,1.08$\pm$0.36,1.16$\pm$0.36,0.55$\pm$0.11,0.73$\pm$0.15,0.78$\pm$0.17
PIF-Item,0.17$\pm$0.11,0.15$\pm$0.12,0.21$\pm$0.12,0.28$\pm$0.08,0.38$\pm$0.17,0.48$\pm$0.23,0.24$\pm$0.12,0.21$\pm$0.12,0.23$\pm$0.12
PIF-Item+Net,0.31$\pm$0.2,0.31$\pm$0.22,0.35$\pm$0.18,0.43$\pm$0.2,0.53$\pm$0.22,0.5$\pm$0.23,0.32$\pm$0.21,0.4$\pm$0.2,0.37$\pm$0.2
PIF-Item+Net+Purchase,0.26$\pm$0.19,0.2$\pm$0.15,0.3$\pm$0.18,0.38$\pm$0.19,0.46$\pm$0.22,0.42$\pm$0.24,0.29$\pm$0.2,0.35$\pm$0.19,0.32$\pm$0.19
PIF-Item (No region),0.1$\pm$0.06,0.11$\pm$0.04,0.13$\pm$0.07,1.15$\pm$0.14,1.65$\pm$0.27,1.7$\pm$0.22,0.3$\pm$0.04,0.35$\pm$0.05,0.38$\pm$0.05


In [ ]:
import pickle
# load up PPC results
with open(res_dir / "dsbmm_ppc_results.pkl", "rb") as f:
    dsbmm_ppc_results = pickle.load(f)
with open(res_dir / "dpf_ppc_results.pkl", "rb") as f:
    dpf_ppc_results = pickle.load(f)
with open(res_dir / "dpf_auc_results.pkl", "rb") as f:
    dpf_auc_results = pickle.load(f)